In [0]:
table_name = "workspace.default.loan_default_highacc"  # change if needed
df = spark.table(table_name)
df.printSchema()
display(df.limit(10))
print("Rows:", df.count())


root
 |-- loan_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- income: long (nullable = true)
 |-- loan_amount: long (nullable = true)
 |-- term: long (nullable = true)
 |-- credit_score: long (nullable = true)
 |-- defaulted: long (nullable = true)



loan_id customer_name income loan_amount term credit_score defaulted LN_1 Rohan Reddy 176542 86321 24 805 0 LN_2 Krishna Gupta 193416 385441 12 800 0 LN_3 Vivaan Pillai 62918 295014 48 793 0 LN_4 Kavya Joshi 72224 206498 24 773 0 LN_5 Rohit Reddy 59298 67772 12 592 0 LN_6 Rohan Bose 100219 444583 36 421 1 LN_7 Ishita Patel 81629 288389 60 519 1 LN_8 Ishita Reddy 86040 101924 12 609 0 LN_9 Nikhil Iyer 48016 578014 36 705 0 LN_10 Saanvi Menon 43960 401296 12 669 0

Rows: 500


In [0]:
from pyspark.sql.functions import col, when

data = (df
    .withColumn("income",        col("income").cast("double"))
    .withColumn("loan_amount",   col("loan_amount").cast("double"))
    .withColumn("term",          col("term").cast("double"))
    .withColumn("credit_score",  col("credit_score").cast("double"))
    .withColumn("label",         col("defaulted").cast("int"))
)

# Derived features
from pyspark.sql.functions import expr
data = (data
    .withColumn("dti",       (col("loan_amount") / (col("income") + 1e-6)))  # Debt-to-Income
    .withColumn("emi_proxy", (col("loan_amount") / (col("term") + 1e-6)))    # rough installment proxy
)

# Drop obvious IDs/text cols for modeling
data = data.drop("loan_id","customer_name","defaulted")

display(data.limit(5))
display(data.groupBy("label").count())


income loan_amount term credit_score label dti emi_proxy 176542.0 86321.0 24.0 805.0 0 0.488954469755135 3596.7081834704923 193416.0 385441.0 12.0 800.0 0 1.9928082474976587 32120.080656659946 62918.0 295014.0 48.0 793.0 0 4.688864871663294 6146.1248719557325 72224.0 206498.0 24.0 773.0 0 2.8591326982324556 8604.082974829877 59298.0 67772.0 12.0 592.0 0 1.1429053256240866 5647.666196027817

label count 0 383 1 117

In [0]:
train_df, test_df = data.randomSplit([0.8, 0.2], seed=42)
print("Train:", train_df.count(), "Test:", test_df.count())


Train: 408 Test: 92


In [0]:
from pyspark.sql.functions import col, lit, when

# train_df must already exist from your randomSplit
# Compute class weights = total / (count_class * num_classes)
counts = train_df.groupBy("label").count().collect()
tot = sum(r["count"] for r in counts)
k = len(counts)
w_map = {int(r["label"]): float(tot)/(float(r["count"])*k) for r in counts}
print("Class weights:", w_map)

# Build the weight expression WITHOUT referencing a 'weight' column
expr = lit(1.0)  # default weight
for lbl, w in w_map.items():
    expr = when(col("label") == lbl, lit(w)).otherwise(expr)

# Add the weight column
train_w = train_df.withColumn("weight", expr)

# Sanity check
train_w.printSchema()
train_w.select("label","weight").groupBy("label").avg("weight").show()


Class weights: {1: 2.125, 0: 0.6538461538461539}
root
 |-- income: double (nullable = true)
 |-- loan_amount: double (nullable = true)
 |-- term: double (nullable = true)
 |-- credit_score: double (nullable = true)
 |-- label: integer (nullable = true)
 |-- dti: double (nullable = true)
 |-- emi_proxy: double (nullable = true)
 |-- weight: double (nullable = false)

+-----+------------------+
|label|       avg(weight)|
+-----+------------------+
|    1|             2.125|
|    0|0.6538461538461582|
+-----+------------------+



In [0]:
from pyspark.ml.feature import VectorAssembler

feature_cols = ["income","loan_amount","term","credit_score","dti","emi_proxy"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

train_vec = assembler.transform(train_w).select("features","label","weight")
test_vec  = assembler.transform(test_df).select("features","label")

train_vec.printSchema()


root
 |-- features: vectorudt (nullable = true)
 |-- label: integer (nullable = true)
 |-- weight: double (nullable = false)



In [0]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol="features", labelCol="label",
                        weightCol="weight", maxIter=100, regParam=0.02)
lr_model = lr.fit(train_vec)
pred_lr = lr_model.transform(test_vec)
display(pred_lr.select("label","prediction","probability").limit(10))


label prediction probability 0 1.0 {"type":"1","size":null,"indices":null,"values":["0.2485820522005197","0.7514179477994802"]} 0 1.0 {"type":"1","size":null,"indices":null,"values":["0.45189506180133193","0.548104938198668"]} 1 1.0 {"type":"1","size":null,"indices":null,"values":["0.07954389782292617","0.9204561021770739"]} 0 0.0 {"type":"1","size":null,"indices":null,"values":["0.8827624856974983","0.11723751430250173"]} 1 1.0 {"type":"1","size":null,"indices":null,"values":["0.028624581718095125","0.9713754182819049"]} 0 0.0 {"type":"1","size":null,"indices":null,"values":["0.9134145692218291","0.08658543077817094"]} 0 0.0 {"type":"1","size":null,"indices":null,"values":["0.9589685104193871","0.04103148958061287"]} 1 1.0 {"type":"1","size":null,"indices":null,"values":["0.1436853066409314","0.8563146933590686"]} 0 0.0 {"type":"1","size":null,"indices":null,"values":["0.6935237554126835","0.3064762445873165"]} 0 1.0 {"type":"1","size":null,"indices":null,"values":["0.26877983137247563","0.7312201686275244"]}

In [0]:
from pyspark.ml.classification import GBTClassifier, RandomForestClassifier

# GBT (binary only; no weightCol)
gbt = GBTClassifier(featuresCol="features", labelCol="label", maxDepth=6, maxIter=80, stepSize=0.1)
gbt_model = gbt.fit(assembler.transform(train_df).select("features","label"))
pred_gbt = gbt_model.transform(test_vec)

# Random Forest (supports imbalance less directly than weights; fine for baseline)
rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=200, maxDepth=10)
rf_model = rf.fit(assembler.transform(train_df).select("features","label"))
pred_rf = rf_model.transform(test_vec)


In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

def eval_binary(pred):
    acc = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy").evaluate(pred)
    f1  = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1").evaluate(pred)
    auc = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC").evaluate(pred)
    aupr= BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderPR").evaluate(pred)
    return acc,f1,auc,aupr

print("LR  :", eval_binary(pred_lr))
print("GBT :", eval_binary(pred_gbt))
print("RF  :", eval_binary(pred_rf))


LR  : (0.8586956521739131, 0.8682752703986273, 0.9738430583501005, 0.929385845599872)
GBT : (1.0, 1.0, 1.0, 1.0)
RF  : (1.0, 1.0, 1.0, 1.0)


In [0]:
chosen = pred_gbt  # or pred_lr if it’s better for your data

pdf = chosen.select("label","prediction").toPandas()
import pandas as pd
cm = pd.crosstab(pdf["label"], pdf["prediction"], rownames=["Actual"], colnames=["Predicted"], margins=True)
cm


Predicted  0.0  1.0  All
Actual                  
0           71    0   71
1            0   21   21
All         71   21   92

In [0]:
# For RF
import numpy as np
imp = rf_model.featureImportances.toArray()
for n,v in sorted(zip(feature_cols, imp), key=lambda x: x[1], reverse=True):
    print(f"{n:15s}  {v:.4f}")


credit_score     0.5216
dti              0.3218
income           0.0714
loan_amount      0.0600
emi_proxy        0.0226
term             0.0026
